In [ ]:
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

dataset_type = ['train', 'val']

def dataset_generator(dataset_type, patch_size, scaling_factor):
    X = []
    y = []

    roots = ["Y2B_23", "Y2B_24"]
    for ds in roots:
        if ds == "Y2B_23":  # Y2B_23 is the only one with train/val subfolders in images
            image_dir = f"../Task_2/datasets/Y2B_23/{ds}/{dataset_type}"
            images_path = sorted(glob.glob(f"{image_dir}/*.png"))
        else:
            image_dir = f"../Task_2/datasets/{ds}/images"
            all_images = glob.glob(f"{image_dir}/*.png")
            images_path = sorted(
                p for p in all_images
                if os.path.basename(p).startswith(f"{dataset_type}_")
            )
        
        mask_root = f"../Task_2/datasets/{ds}/masks"

        for image_path in images_path:
            base = os.path.basename(image_path)
            stem, _ = os.path.splitext(base)

            if ds == "Y2B_23":
                root_candidates = os.path.join(mask_root, stem + "_root_mask.tif")
                if not root_candidates:
                    continue
            else:
                root_candidates = glob.glob(os.path.join(mask_root, "*", stem + "_root_mask.tif"))
                if not root_candidates:
                    continue
                root_mask_path = root_candidates[0]

            if not os.path.exists(root_mask_path):
                raise FileNotFoundError(f"Mask not found: {root_mask_path}")
            
            img = cv2.imread(image_path)
            root_mask = cv2.imread(root_mask_path, cv2.IMREAD_GRAYSCALE)

            if scaling_factor != 1:
                img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor)
                root_mask = cv2.resize(root_mask, None, fx=scaling_factor, fy=scaling_factor)
                
            height, width = img.shape[:2]

            for top in range(0, height - patch_size + 1, patch_size):
                for left in range(0, width - patch_size + 1, patch_size):
                    img_patch = img[top:top+patch_size, left:left+patch_size, :] # x, y, chanels
                    root_patch = root_mask[top:top+patch_size, left:left+patch_size]

                    root_patch = root_patch[..., np.newaxis] # "..." - Ellipsis - takes all dimensions

                    X.append(img_patch)
                    y.append(root_patch)

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    
    # Normalize
    X = X/255
    y = y/255
    return X, y

In [ ]:
patch_size = 128
scaling_factor = 1

In [ ]:
X_train, y_train = dataset_generator(dataset_type='train',
                                     patch_size=patch_size,
                                     scaling_factor=scaling_factor)
X_train.shape, y_train.shape # ((4680, 128, 128, 3), (4680, 128, 128, 1))

In [ ]:
X_val, y_val = dataset_generator(dataset_type='val',
                                 patch_size=patch_size,
                                 scaling_factor=scaling_factor)
X_val.shape, y_val.shape # ((520, 128, 128, 3), (520, 128, 128, 1))

In [ ]:
# This script needs these libraries to be installed:
#   tensorflow, numpy

import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

import random
import numpy as np
import tensorflow as tf


# Start a run, tracking hyperparameters
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata with wandb.config
    config={
        "layer_1": 256,
        "activation_1": "relu",
        "dropout": random.uniform(0.01, 0.80),
        "layer_2": 10,
        "activation_2": "softmax",
        "optimizer": "sgd",
        "loss": "sparse_categorical_crossentropy",
        "metric": "accuracy",
        "epoch": 8,
        "batch_size": 256
    }
)

# [optional] use wandb.config as your config
config = wandb.config

# get the data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train, y_train = x_train[::5], y_train[::5]
x_test, y_test = x_test[::20], y_test[::20]
labels = [str(digit) for digit in range(np.max(y_train) + 1)]

# build a model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
    tf.keras.layers.Dropout(config.dropout),
    tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
    ])

# compile the model
model.compile(optimizer=config.optimizer,
              loss=config.loss,
              metrics=[config.metric]
              )

# WandbMetricsLogger will log train and validation metrics to wandb
# WandbModelCheckpoint will upload model checkpoints to wandb
history = model.fit(x=x_train, y=y_train,
                    epochs=config.epoch,
                    batch_size=config.batch_size,
                    validation_data=(x_test, y_test),
                    callbacks=[
                      WandbMetricsLogger(log_freq=5),
                      WandbModelCheckpoint("models")
                    ])

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()